<a href="https://colab.research.google.com/github/amankiitg/Agents/blob/main/Blog_writer_using_CrewAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create Multi-AI Agent to Research and Write an Article


Learn about Agents, Tasks and organize agents to work sequentially.

In [1]:
# To install crewAI, you need to have Python >=3.10 and <=3.13 installed on your system:
import sys
print(sys.version)

3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]


### Step 01. Intalling dependencies

Installing crewAi library and crewAI tools

In [2]:
pip install crewai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.5/119.5 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 6.9 MB/s eta 0

In [3]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

### Step 02. Importing Libraries

Installing crewAi library and crewAI tools

In [4]:
from crewai import Agent, Task, Crew

For LLM agents, we will be using OpenAI's gpt-3.5-turbo.

In [5]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('openaikey')

### Step 03. Creating Agents

Create instance of the Agent class with the desired properties, `role`, `goal` and `backstory`

- `role`: agent's function within the crew
- `goal`: agent's objective. Guides agent's decision making process
- `backstory`: provides context to agent's role and goal. Helps in interation and collaboration.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [6]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic",
    allow_delegation=False,
    verbose=True
)

### Agent: Writer

In [7]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [8]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

### Step 04. Define the Tasks

Tasks are **specific assignments**, completed by agents.
They **provide all necessary details for execution**, such as a `description`, `expected_output` , `agent` responsible, tools, etc.

- `description`: Tells about what task entails
- `expected_output`: Tells about what task output should look like
- `agent`: Agent responsible for the task

### Task: Plan

In [9]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [10]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
        "3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer
)

### Task: Edit

In [11]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

### Step 05. Forming the crew

Crew is **group of AI agents**, collaborating to perform set of tasks.


- `tasks`: list of tasks
- `agents`: list of agents
- `process (optional)`: work flow (e.g., sequential, hierarchical) default : sequential
- `verbose=2` allows you to see all the logs of the execution.


    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.



In [12]:
crew = Crew(
    tasks=[plan, write, edit],
    agents=[planner, writer, editor],
    verbose=False
)

### Step 06.  Kick It Off

Pass in a topic of your choice and see what the agents come up with!

In [13]:
topic = "Artificial Intelligence"
result = crew.kickoff(inputs={"topic": topic})

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.            │
│  2. Identify the target audience, considering their interests and pain points.                                  │
│  3. Develop a detailed content outline including an introduction, key points, and a call to action.             │
│  4. Include SEO keywords and relevant data or sources.                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # Comprehensive Content Plan for Blog Article on Artificial Intelligence                                       │
│                                                                                                                 │
│  ## Target Audience Analysis                                                                                    │
│  ### Target Audience                                                                                            │
│  1. **Tech Professionals**                                                                                      │
│     - Interests: Latest technologies, innovations in AI, machine learning, and automation.                      │
│     - Pain Points: Keeping up with rapid technological advancements, understanding AI principles, and           │
│  implementing AI solutions.                                                                                     │
│                                                                                                                 │
│  2. **Business Leaders**                                                                                        │
│     - Interests: How AI can enhance business processes, improve efficiency, and drive innovation.               │
│     - Pain Points: Identifying effective AI strategies, potential ROI on AI investments, and managing change.   │
│                                                                                                                 │
│  3. **Students & Tech Enthusiasts**                                                                             │
│     - Interests: Learning about AI, potential career paths in AI, and understanding the societal impacts of     │
│  technology.                                                                                                    │
│     - Pain Points: Finding trustworthy resources for learning, understanding complex topics, and navigating     │
│  the job market.                                                                                                │
│                                                                                                                 │
│  4. **General Public**                                                                                          │
│     - Interests: The implications of AI on everyday life, ethical considerations, and how AI technology can     │
│  affect employment.                                                                                             │
│     - Pain Points: Misinformation about AI, fear regarding job displacement, and understanding AI's potential   │
│  benefits.                                                                                                      │
│                                                                                                                 │
│  ## Latest Trends and Key Players                                                                               │
│  ### Trends                                                                                                     │
│  1. **AI in Healthcare**: The use of AI for diagnostics, patient care, and personalized medicine is on the      │
│  rise.                                                                                                          │
│  2. **Generative AI**: Powerful models like OpenAI's GP

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Writer                                                                                          │
│                                                                                                                 │
│  Task: 1. Use the content plan to craft a compelling blog post on Artificial Intelligence.                      │
│  2. Incorporate SEO keywords naturally.                                                                         │
│  3. Sections/Subtitles are properly named in an engaging manner.                                                │
│  4. Ensure the post is structured with an engaging introduction, insightful body, and a summarizing             │
│  conclusion.                                                                                                    │
│  5. Proofread for grammatical errors and alignment with the brand's voice.                                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Writer                                                                                          │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```markdown                                                                                                    │
│  # The Future of Artificial Intelligence: Trends, Ethical Considerations, and Key Players                       │
│                                                                                                                 │
│  ## Introduction                                                                                                │
│  Artificial Intelligence (AI) is no longer confined to the realm of science fiction; it permeates various       │
│  aspects of our daily lives, from healthcare advancements to the automation of business processes. Its ability  │
│  to learn and adapt has transformed industries, reshaping the way we work and interact with technology.         │
│  Understanding current trends in AI is essential not just for tech professionals and business leaders, but      │
│  also for the general public, as the implications of AI technology touch everyone's lives.                      │
│                                                                                                                 │
│  It is crucial to stay informed about the latest developments in AI, as they can create significant             │
│  opportunities and challenges. This article will explore the defining characteristics of AI, delve into         │
│  current trends, highlight key players in the field, and address the ethical considerations accompanying these  │
│  advancements.                                                                                                  │
│                                                                                                                 │
│  ## Defining Artificial Intelligence                                                                            │
│  Artificial Intelligence encompasses a collection of technologies designed to simulate human-like cognitive     │
│  functions, such as learning, reasoning, and problem-solving. At the core of AI are key concepts like Machine   │
│  Learning (ML) and Deep Learning, which enable systems to learn from data and improve without explicit          │
│  programming. Machine Learning involves algorithms that analyze data, identify patterns, and make decisions     │
│  based on that information, while Deep Learning, a subset of ML, uses neural networks to process vast amounts   │
│  of unstructured data.                                                                                          │
│                                                                                                                 │
│  As AI technology continues to evolve, its capabilities expand beyond simple task automation to more complex    │
│  problem-solving applications across sectors. Understanding these foundational concepts is vital as they form   │
│  the basis of numerous AI-driven solutions that businesses and individuals utilize today.                       │
│                                                                                                                 │
│  ## Current Trends in AI                                                                                        │
│  One of the most notable trends in AI today is its application in healthcare, where technologies are being      │
│  leveraged for diagnostics, patient care optimization, 

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│  Task: Proofread the given blog post for grammatical errors and alignment with the brand's voice.               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```markdown                                                                                                    │
│  # The Future of Artificial Intelligence: Trends, Ethical Considerations, and Key Players                       │
│                                                                                                                 │
│  ## Introduction                                                                                                │
│  Artificial Intelligence (AI) is no longer confined to the realm of science fiction; it permeates various       │
│  aspects of our daily lives, from healthcare advancements to the automation of business processes. Its ability  │
│  to learn and adapt has transformed industries, reshaping the way we work and interact with technology.         │
│  Understanding current trends in AI is essential not just for tech professionals and business leaders, but      │
│  also for the general public, as the implications of AI technology touch everyone's lives.                      │
│                                                                                                                 │
│  It is crucial to stay informed about the latest developments in AI, as they can create significant             │
│  opportunities and challenges. This article will explore the defining characteristics of AI, delve into         │
│  current trends, highlight key players in the field, and address the ethical considerations accompanying these  │
│  advancements.                                                                                                  │
│                                                                                                                 │
│  ## Defining Artificial Intelligence                                                                            │
│  Artificial Intelligence encompasses a collection of technologies designed to simulate human-like cognitive     │
│  functions, such as learning, reasoning, and problem-solving. At the core of AI are key concepts like Machine   │
│  Learning (ML) and Deep Learning, which enable systems to learn from data and improve without explicit          │
│  programming. Machine Learning involves algorithms that analyze data, identify patterns, and make decisions     │
│  based on that information, while Deep Learning, a subset of ML, uses neural networks to process vast amounts   │
│  of unstructured data.                                                                                          │
│                                                                                                                 │
│  As AI technology continues to evolve, its capabilities expand beyond simple task automation to more complex    │
│  problem-solving applications across sectors. Understanding these foundational concepts is vital as they form   │
│  the basis of numerous AI-driven solutions that businesses and individuals utilize today.                       │
│                                                                                                                 │
│  ## Current Trends in AI                                                                                        │
│  One of the most notable trends in AI today is its application in healthcare, where technologies are being      │
│  leveraged for diagnostics, patient care optimization, 

In [14]:
from IPython.display import Markdown

# Extract markdown content from between the code blocks
markdown_content = result.raw.split('```markdown\n')[1].split('\n```')[0]
Markdown(markdown_content)

# The Future of Artificial Intelligence: Trends, Ethical Considerations, and Key Players  

## Introduction  
Artificial Intelligence (AI) is no longer confined to the realm of science fiction; it permeates various aspects of our daily lives, from healthcare advancements to the automation of business processes. Its ability to learn and adapt has transformed industries, reshaping the way we work and interact with technology. Understanding current trends in AI is essential not just for tech professionals and business leaders, but also for the general public, as the implications of AI technology touch everyone's lives.

It is crucial to stay informed about the latest developments in AI, as they can create significant opportunities and challenges. This article will explore the defining characteristics of AI, delve into current trends, highlight key players in the field, and address the ethical considerations accompanying these advancements.  

## Defining Artificial Intelligence  
Artificial Intelligence encompasses a collection of technologies designed to simulate human-like cognitive functions, such as learning, reasoning, and problem-solving. At the core of AI are key concepts like Machine Learning (ML) and Deep Learning, which enable systems to learn from data and improve without explicit programming. Machine Learning involves algorithms that analyze data, identify patterns, and make decisions based on that information, while Deep Learning, a subset of ML, uses neural networks to process vast amounts of unstructured data.

As AI technology continues to evolve, its capabilities expand beyond simple task automation to more complex problem-solving applications across sectors. Understanding these foundational concepts is vital as they form the basis of numerous AI-driven solutions that businesses and individuals utilize today.  

## Current Trends in AI  
One of the most notable trends in AI today is its application in healthcare, where technologies are being leveraged for diagnostics, patient care optimization, and personalized medicine. For instance, AI-driven algorithms are capable of analyzing medical images with a high degree of accuracy, improving diagnosis rates of diseases like cancer. Case studies have showcased hospitals successfully utilizing AI for patient management, leading to better overall health outcomes.

Generative AI is another breakthrough, with platforms like OpenAI's GPT and DALL-E redefining creative industries. These models allow for the automatic generation of text and images, making it easier for content creators and marketers to produce engaging materials. The ongoing integration of AI technologies into business automation also enhances productivity, allowing organizations to streamline operations and reduce costs.  

## Key Players in the AI Space  
Several companies are at the forefront of AI innovation, shaping the landscape with their groundbreaking contributions. OpenAI stands out as a pioneer in generative AI and natural language processing, developing systems that engage users in human-like dialogue. Google AI follows closely, driving advancements in various applications that improve user experience and operational efficiency across different platforms.

IBM Watson is increasingly recognized for its AI applications in sectors like healthcare and finance, demonstrating the utility of analytics in decision-making processes. In addition, Microsoft's Azure AI empowers businesses with cloud-based AI solutions, fostering an ecosystem that allows companies to safely adopt AI technology. Emerging startups, armed with innovative ideas, are crucial in injecting fresh energy into the AI sector and proving that the future of this technology remains dynamic and promising.  

## Ethical Considerations in AI  
With great power comes great responsibility, and AI technology is no exception. The ethical implications surrounding AI deployment are becoming a pressing concern as biases in algorithms can lead to skewed decisions, particularly in sensitive areas like hiring, law enforcement, and lending. Addressing these biases is essential, and many experts advocate for transparency and accountability in AI systems. 

Moreover, privacy considerations are paramount as AI systems often rely on large datasets containing personal information. Striking the right balance between leveraging data for AI applications and ensuring user privacy poses a significant challenge. It is critical for professionals and companies to navigate this landscape with care to foster a responsible and ethical AI deployment.  

## Conclusion  
Artificial Intelligence continues to shape the future, offering promising innovations while also presenting ethical challenges that require careful consideration. As technology evolves, it is essential for professionals, businesses, and the general public to engage with AI responsibly. By understanding its trends, recognizing key players, and addressing ethical implications, we can harness the power of AI for the greater good.

We encourage readers to explore AI learning resources, subscribe for updates on AI trends, and consider the ethical implications of AI technologies in their own work. As we embrace ongoing advancements in this field, let's cultivate an informed and responsible approach to AI.  

In [ ]:
!pip freeze > requirements.txt